Osnabrück University - Machine Learning (Summer Term 2016) - Prof. Dr.-Ing. G. Heidemann, Ulf Krumnack

# Exercise Sheet 02: Decision Trees

## Assignment 1: Decision Trees
Draw the decision trees for the following boolean functions. Either use pen and paper or employ your ASCII artist within below. 

Note: $\oplus := xor$, that means one of the operands has to be true, while the other one has to be false:

$\oplus$ | $B$ | $\neg B$
---------|-----|---------
$A$      |  f  |    t
$\neg A$ |  t  |    f

### a) $\neg A \wedge B$

### b) $A \oplus B$ 

### c) $A \vee (B \wedge C) \vee (\neg C \wedge D)$

### d) $(A \rightarrow (B \wedge \neg C)) \vee (A \wedge B)$

$= (\neg A \vee (B \wedge \neg C)) \vee (A \wedge B)$

$= \neg A \vee (A \wedge B) \vee (B \wedge \neg C)$

$= \neg A \vee (A \wedge B)$

## Assignment 2: Entropy and Information Gain

Attributes and their possible values:

  * $raining = \{yes, no\}$
  * $tired = \{yes, no\}$
  * $late = \{yes, no\}$
  * $distance = \{short, medium, long\}$

Training examples:

| #  | raining | tired | late | distance | attend_party |
|----|---------|-------|------|----------|--------------|
| 1  | yes | no  | no  | short  | **yes** |
| 2  | yes | no  | yes | medium | **no**  |
| 3  | no  | yes | no  | long   | **no**  |
| 4  | yes | yes | yes | short  | **no**  |
| 5  | yes | no  | no  | short  | **yes** |
| 6  | no  | no  | no  | medium | **yes** |
| 7  | no  | yes | no  | long   | **no**  |
| 8  | yes | no  | yes | short  | **no**  |
| 9  | yes | yes | no  | short  | **yes** |
| 10 | no  | yes | no  | medium | **no**  |
| 11 | no  | yes | no  | long   | **no**  |
| 12 | no  | yes | yes | short  | **no**  |

### a)
Build the root node of a decision tree from the training examples given in the table above by calculate the information gain on all four attributes (raining, tired, late, distance).

$$Gain(S,A) = Entropy(S) - \sum_{v \in Values(A)} \frac{|S_v|}{|S|}Entropy(S_v)$$
$$Entropy(S) = -p_{\oplus} log_{2} p_{\oplus} - p_{\ominus} log_{2} p_{\ominus}$$



$S_v$ is the subset of $S$ for which attribute A has value v. Example for attribute *tired*:
$S_{yes} \leftarrow [1+, 6−], |S_{yes}| = 7 \quad \quad \quad S_{no} \leftarrow [3+, 2−], |S_{no}| = 5$

### b)
Perform the same calculation as in **a)** but use the gain ratio instead of the information gain. Does the result for the root node change?

$$GainRatio(S,A) = \frac{Gain(S,A)}{SplitInformation(S,A)}$$
$$SplitInformation(S,A) = - \sum_{v \in Values(A)} \frac{|S_v|}{|S|} log_{2} \frac{S_{v}}{S}$$

### c)
Let’s assume the root node is a node which checks the value of the attribute has *distance*. Calculate the next level of the decision tree using the information gain.

## Assignment 3
Implement the following two functions in Python. Take a look at the `assert`s to see how the function should behave. An assert is a condition that your function is required to pass. Most of the conditions here are taken from the lecture slides (ML-03, Slide 12 & 13). Don't worry if you do not get all asserts to pass, just comment them out.

### a) Entropy
$$Entropy(S) = - \sum_{i=1...c} p_i log_2 p_i$$

In [ ]:
from math import log
def entropy(S):
    """
    Calculate the entropy for a given target value set. 
    S: List of target classes for specific observations.
    """
    p_i = [len([value for value in S if value == c])/len(S) for c in set(S)]
    return - sum(p * log(p, 2) for p in p_i) 


# See ML-03, Slide 12 & 13
assert entropy([1,1,1,0,0,0]) == 1.0
assert round(entropy([1,1,1,1,0,0,0]), 3) == 0.985
assert round(entropy([1,1,1,1,1,1,0]), 3) == 0.592
assert round(entropy([1,1,1,1,1,1,0,0]), 3) == 0.811
assert round(entropy([2,2,1,1,0,0]), 3) == 1.585
assert round(entropy([2,2,2,1,0]), 3) == 1.371
assert round(entropy([2,2,2,0,0]), 3) == 0.971

### b)  Information Gain
$$Gain(S,A) = Entropy(S) - \sum_{v \in Values(A)} \frac{|S_v|}{|S|} Entropy(S_v)$$

In [ ]:
def gain(S, A):
    """
    Calculates the expected reduction in entropy due to sorting on A.
    S: Target classes for observations in A.
    A: Observations.
    """
    sigma = 0
    for v in set(A): # sets only contain unique values
        S_v = [S[key] for (key, v_) in enumerate(A) if v_ == v]
        sigma = sigma + ((len(S_v) / len(S)) * entropy(S_v))
    return entropy(S) - sigma


# See ML-03, Slide 12 & 13
assert_S_ = [0,0,1,1,1,0,1,0,1,1,1,1,1,0]
assert round(gain(assert_S_, [1,1,1,1,0,0,0,1,0,0,0,1,0,1]), 3) == .152
assert round(gain(assert_S_, [0,1,0,0,0,1,1,0,0,0,1,1,0,1]), 3) == .048

### c) ID3 [0p]
Nothing to do here currently.

In [ ]:
from collections import Counter
from node import Node

def id3(examples, attributes, target_attribute = None): 
    """
    Calculate a tree of Nodes (fields: label [string], children [list]) 
    using the ID3 algorithm found as pseudocode on Wikipedia. Including comments (currently).
    
    TODO: Needs a second pair of eyes and comments clean up. Comments kept for Wikipedia pseudocode reference.
    """
    # Create a root node for the tree
    # If all examples are positive, Return the single-node tree Root, with label = 
    # If all examples are negative, Return the single-node tree Root, with label = -.
    if all(examples['targets'] == examples['targets'][0]):
        return Node(examples['targets'][0], [])
    
    # If number of predicting attributes is empty, then Return the single node tree Root,
    # with label = most common value of the target attribute in the examples.
    if len(attributes) == 0:
        return Node(Counter(data_sample[target_attribute] for data_sample in examples['data']).most_common(1), [])
    
    # A <-- The Attribute that best classifies examples.
    gains = [gain(examples['targets'], [r[attribute] for r in examples['data']]) for attribute in attributes]
    attribute = attributes[gains.index(min(gains))]

    # Create a root node for the tree (2)
    # Decision Tree attribute for Root = A.
    root = Node(attribute, [])
    
    # Otherwise Begin
    # For each possible value, vi, of A,
    for vi in set(data_sample[attribute] for data_sample in examples['data']):
        # Add a new tree branch below Root, corresponding to the test A = vi.
        child = Node(vi, [])
        root.children.append(child)
        # Let Examples(vi) be the subset of examples that have the value vi for A
        vi_indices = [idx for idx, data_sample in enumerate(examples['data']) if data_sample[attribute] == vi]
        examples_vi = {
            'data': [examples['data'][i] for i in vi_indices],
            'targets': [examples['targets'][i] for i in vi_indices]
        }

        # If Examples(vi) is empty
        if len(examples_vi['data']) == 0:
            # Then below this new branch add a leaf node with label = most common target value in the examples
            child.children.append(Counter(examples_vi['targets']).most_common(1))
        else:
            # Else below this new branch add the subtree ID3 (Examples(vi), Attributes – {A})
            child.children.append(
                id3(examples_vi, \
                    [attribute_ for attribute_ in attributes if not attribute_ == attribute], \
                    attribute)
            )
    # End

    # Return root
    return root

### c2) Apply ID3 algorithm.
Again, nothing to do here currently for students.

In [ ]:
# Example dataset. See Wikipedia (add URL here).
from sklearn.datasets import load_iris
iris = load_iris()

# Make sure our gain function handles the new dataset as expected.
assert round(gain(iris.target, [r[2] for r in iris.data]), 3) == 1.446

attributes = [idx for idx, _ in enumerate(iris.data[0])]
tree = id3({'data': iris.data, 'targets': iris.target}, attributes)

print(tree)